### Notebook to analyze IFWASTE-Simulation results

In [1]:
import os 
import pandas as pd 
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

In [2]:
#PATH = "/blue/carpena/haasehelen/ifwaste/data/"
#CONFIG_PATH = "/blue/carpena/haasehelen/ifwaste/model/config.json"

In [3]:
PATH = "E:/UF/ifwaste/data/"
CONFIG_PATH = "E:/UF/ifwaste/model/config.json"

### Loading necessary parameters from config file
- weights per serving of each food category

In [ ]:
FGMEAT_KG = FGDAIRY_KG = FGBAKED_KG = FGVEGETABLE_KG = FGDRYFOOD_KG = FGSNACKS_KG = FGSTOREPREPARED_KG = None
with open(CONFIG_PATH) as f:
    config = json.load(f)
    FGMEAT_KG = config["Food"]["FGMeat"]["kg_per_serving"]
    FGDAIRY_KG = config["Food"]["FGDairy"]["kg_per_serving"]
    FGBAKED_KG = config["Food"]["FGBaked"]["kg_per_serving"]
    FGVEGETABLE_KG = config["Food"]["FGVegetable"]["kg_per_serving"]
    FGDRYFOOD_KG = config["Food"]["FGDryFood"]["kg_per_serving"]
    FGSNACKS_KG = config["Food"]["FGSnacks"]["kg_per_serving"]
    FGSTOREPREPARED_KG = config["Food"]["FGStorePrepared"]["kg_per_serving"]
    


In [5]:
lookup_df = pd.DataFrame({
    'Type': ['Meat & Fish', 'Dairy & Eggs', 'Fruits & Vegetables', 'Dry Foods', 'Baked Goods' ,
    'Snacks, Condiments, Oils & Other', 'Store-Prepared Items'],
    'Servings_to_Kg': [FGMEAT_KG, FGDAIRY_KG, FGVEGETABLE_KG, FGDRYFOOD_KG, FGBAKED_KG, FGSNACKS_KG, FGSTOREPREPARED_KG],
    "Color": ["#116A65", "#00a0e1", "#466eb4", "#e6a532", "#d7642c","#73B55B", "#D82E5E"]
})

status_colors = {
            "Inedible Parts": "#26547C",
            "Plate Waste": "#FFD166",
            "Spoiled Food": "#EF476F"
        }
    
#color_mapping = dict(zip(lookup_df['Type'], lookup_df['Color']))

In [6]:
def load_data() -> dict[str, dict[str, dd.DataFrame]]:
    # Dictionary to store DataFrames for each run
    runs_data = {}

    # List of the main folders to process
    main_folders = [f for f in os.listdir(PATH) if os.path.isdir(os.path.join(PATH, f))]
    # Iterate over each main folder
    for main_folder in main_folders:
        folder_path = os.path.join(PATH, main_folder)  # type: ignore
        
        # Iterate over each run folder inside the main folder
        for run_folder in os.listdir(folder_path):
            run_path = os.path.join(folder_path, run_folder)
            if os.path.isdir(run_path):
                run_id = run_folder  # Use the folder name as the run ID (e.g., 'run_0')
                # Initialize a dictionary to store DataFrames for each log file in this run
                run_logs = {}
                # Get all CSV files in the run folder
                file_names = [
                    f for f in os.listdir(run_path)
                    if os.path.isfile(os.path.join(run_path, f)) and f.endswith('.csv')
                ]
                
                # Read each CSV file into a Dask DataFrame
                for file in file_names:
                    file_path = os.path.normpath(os.path.join(run_path, file))
                    if os.path.exists(file_path):
                        log_name = file[:-4]  # Remove '.csv' to get the log name
                        run_logs[log_name] = dd.read_csv(file_path,assume_missing=True) #assume all values as floats
                
                # Add the logs dictionary to the runs_data under the current run ID
                runs_data[run_id] = run_logs

    return runs_data


In [7]:
import os

def load_merged_data() -> dict[str, dd.DataFrame]:
    file_names = ['log_bought', 'log_eaten', 'log_wasted', 'log_hh_config', "log_still_have",
                  "log_hh_daily", "log_sim_config", "log_store_daily", "log_wasted"]
    
    # Dictionary to store merged DataFrames for each CSV type across all runs
    data_dict = {}

    # List of the main folders to process
    main_folders = [f for f in os.listdir(PATH) if os.path.isdir(os.path.join(PATH, f))]
    
    # Iterate over each main folder
    for main_folder in main_folders:
        folder_path = os.path.join(PATH, main_folder)
        
        # Iterate over each run folder inside the main folder
        for run_folder in os.listdir(folder_path):
            run_path = os.path.join(folder_path, run_folder)
            if os.path.isdir(run_path):
                run_id = run_folder.split('_')[-1]  # Extract the run ID from the folder name

                # Iterate over each CSV type
                for csv_type in file_names:
                    file_path = os.path.join(run_path, f'{csv_type}.csv')
                    if os.path.exists(file_path):
                        # Read the CSV file into a Dask DataFrame
                        df = dd.read_csv(file_path, assume_missing=True)
                        
                        # Modify the 'household' column if it exists
                        if 'household' in df.columns:
                            df['household'] = run_id + "_" + df['household'].astype(int).astype(str)
                        if "Unnamed: 0" in df.columns: 
                            df = df.drop(columns=["Unnamed: 0"])
                        
                        # Rename columns that have "Day" in their name to just "Day"
                        for item in df.columns: 
                            if "Day" in item: 
                                df = df.rename(columns={item: "Day"})
                        
                        # Append the DataFrame to the corresponding entry in data_dict
                        if csv_type not in data_dict:
                            data_dict[csv_type] = df
                        else:
                            # Concatenate with the existing DataFrame
                            data_dict[csv_type] = dd.concat([data_dict[csv_type], df], axis=0)

    return data_dict


In [8]:
merged_data = load_merged_data()
data = load_data()

In [9]:
d = merged_data["log_bought"].compute()
d

,household,day,type,servings,days_till_expiry,price_per_serving,sale_type,discount_effect,amount,sale_timer,store,product_ID
0,0_76,0.0,FGDAIRY,6.0,11.0,1.800000,EnumSales.NONE,EnumDiscountEffect.NONE,11.0,1000.0,"Premium_retailer at (4, 6)",FGDAIRY61.8
1,0_76,0.0,FGDRYFOOD,6.0,21.0,0.500000,EnumSales.NONE,EnumDiscountEffect.NONE,25.0,1000.0,"Premium_retailer at (4, 6)",FGDRYFOOD60.5
2,0_76,0.0,FGSNACKS,3.0,20.0,2.000000,EnumSales.NONE,EnumDiscountEffect.NONE,12.0,1000.0,"Premium_retailer at (4, 6)",FGSNACKS32.0
3,0_76,0.0,FGVEGETABLE,6.0,14.0,1.300000,EnumSales.NONE,EnumDiscountEffect.NONE,3.0,1000.0,"Premium_retailer at (4, 6)",FGVEGETABLE61.3
4,0_33,0.0,FGBAKED,10.0,11.0,1.000000,EnumSales.NONE,EnumDiscountEffect.NONE,2.0,1000.0,"Premium_retailer at (4, 6)",FGBAKED101.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1598,5_40,9.0,FGSTOREPREPARED,24.0,4.0,0.250000,EnumSales.SEASONAL,EnumDiscountEffect.BOGO,1.0,5.0,"Discount_retailer at (6, 3)",FGSTOREPREPARED121.0
1599,5_10,9.0,FGDRYFOOD,6.0,31.0,0.350000,EnumSales.SEASONAL,EnumDiscountEffect.DISCOUNT30,1.0,4.0,"Convenience_store at (3, 5)",FGDRYFOOD60.5
1600,5_10,9.0,FGSTOREPREPARED,6.0,1.0,0.666667,EnumSales.EXPIRING,EnumDiscountEffect.DISCOUNT60,1.0,1000.0,"Convenience_store at (3, 5)",FGSTOREPREPARED61.0
1601,5_10,9.0,FGSTOREPREPARED,12.0,5.0,1.000000,EnumSales.NONE,EnumDiscountEffect.NONE,1.0,1000.0,"Convenience_store at (3, 5)",FGSTOREPREPARED121.0


In [10]:
# Number of houses
N_HH = data["run_0"]["log_hh_config"]["household"].max().compute() + 1

In [11]:
sim_config = pd.DataFrame(data["run_0"]["log_sim_config"])
DAYS = sim_config[0][0]

In [12]:
EXPERIMENT_KEYS = data.keys()

## Sanity Checks: 
1. Biomass check


In [ ]:
for key in EXPERIMENT_KEYS: 
    bought = data[key]["log_bought"]["servings"] * data[key]["log_bought"]["amount"]
    bought = bought.sum().compute()
    
    eaten = data[key]["log_eaten"]["servings"].sum().compute()
    wasted = data[key]["log_wasted"]["servings"].sum().compute()
    left = data[key]["log_still_have"]["servings"].sum().compute()
    
    print("group: ", key)
    print("bought total servings:\t\t", int(bought))
    print("eaten total servings:\t\t", int(eaten))
    print("wasted total servings:\t\t", int(wasted))    
    print("left last day in servings:\t", int(left))
    
    diff = bought - (eaten + wasted + left)

    print("bought - consumed:\t\t", diff)
    print("------------")

group:  run_0
bought total servings:		 33431
eaten total servings:		 27350
wasted total servings:		 10880
left last day in servings:	 4888
bought - consumed:		 -9687.902123476357
------------
group:  run_1
bought total servings:		 32694
eaten total servings:		 26295
wasted total servings:		 9444
left last day in servings:	 5807
bought - consumed:		 -8853.165561652131
------------
group:  run_2
bought total servings:		 34323
eaten total servings:		 27109
wasted total servings:		 7751
left last day in servings:	 6714
bought - consumed:		 -7252.448590026484
------------
group:  run_3
bought total servings:		 33622
eaten total servings:		 27932
wasted total servings:		 8807
left last day in servings:	 6278
bought - consumed:		 -9396.487380695005
------------
group:  run_4
bought total servings:		 35437
eaten total servings:		 27929
wasted total servings:		 8899
left last day in servings:	 6732
bought - consumed:		 -8125.054325497367
------------
group:  run_5
bought total servings:		 33142

In [29]:
#what all houes of run 0 should eat in days
should_eat = data["run_0"]["log_hh_config"]["required_servings"].sum().compute() * DAYS
should_eat

33704.2775768385

In [30]:
#how much servings are missing over all hh of run 0 over days
missing_eat = data["run_0"]["log_hh_daily"]["servings"].sum().compute()
missing_eat


3234.1433984014416

In [15]:
merged_data.keys()

dict_keys(['log_bought', 'log_eaten', 'log_wasted', 'log_hh_config', 'log_still_have', 'log_hh_daily', 'log_sim_config', 'log_store_daily'])

In [32]:
eaten = should_eat - missing_eat
eaten

30470.134178437063

### Preprocess data for per capita and per household baseline values
have a second datastructure in place that has all the hh merged together for some analysis like average waste per household. 

data <- holds data organized by run

data_merged <- holds data organized as 1 big simulation, hh unique through HH+RUN 



In [16]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

def preprocess_data(dask_df, per_capita):
    # Extract Dask DataFrames
    print(dask_df.keys())
    config_df = dask_df["log_hh_config"].compute()
    wasted = dask_df["log_wasted"]
    
    # Merge data with configuration to get household information
    merged_df = dd.merge(wasted, config_df[['household', 'adults', 'children']], on='household', how='inner')
    
    # Calculate the total number of people
    total_people = 1
    if per_capita:
        total_people = config_df['adults'][0].values[0] + config_df['children'][0].values[0]
    # Group by day and household, then sum waste and normalize if per capita
    df_grouped = merged_df.groupby(by=['day', 'household'])['servings'].sum().compute()
    df_grouped = df_grouped / total_people

    # Convert to a DataFrame and reset index
    df_grouped = df_grouped.reset_index()

    # Create a DataFrame with all days and houses
    all_houses = merged_df['household'].unique().compute()
    print(all_houses)
    all_combinations = pd.MultiIndex.from_product([range(DAYS), all_houses], names=['day', 'household'])
    
    # Reindex to include all days and houses, filling missing values with 0
    df_complete = df_grouped.set_index(['day', 'household']).reindex(all_combinations, fill_value=0).reset_index()
    print(df_complete.head())
    print(df_complete["servings"].std())
    print(df_complete["servings"].mean())
    # Convert back to a Dask DataFrame
    df_complete = dd.from_pandas(df_complete, npartitions=1)
    

    return df_complete


In [17]:
prep_0k = preprocess_data(merged_data,True)
prep_0k.head

dict_keys(['log_bought', 'log_eaten', 'log_wasted', 'log_hh_config', 'log_still_have', 'log_hh_daily', 'log_sim_config', 'log_store_daily'])


c:\Users\helen\miniconda3\envs\ifwaste-env\Lib\site-packages\dask\dataframe\multi.py:520: UserWarning: Merging dataframes with merge column data type mismatches: 
+----------------------------+------------+-------------+
| Merge columns              | left dtype | right dtype |
+----------------------------+------------+-------------+
| ('household', 'household') | object     | string      |
+----------------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


0      0_5
1     0_25
2     0_43
3     0_11
4      0_3
      ... 
25    1_36
26    2_23
27    2_27
28    2_48
29    2_19
Name: household, Length: 150, dtype: object


TypeError: 'numpy.float64' object cannot be interpreted as an integer